Online state estimation for random walk example
===
This demo implements the state estimation for a random walk with Gaussian noise. Along the way we will use the `wrap` concept and make use of write buffers. 

Let our estimation of the state $X[n]$ be a univariate Gaussian distribution with mean $m$ and variance $V$ at time index $n$. At each consecutive timestep we update $X[n]$ by adding a constant Gaussian noise `N`, with mean `m_noise` and variance `V_noise`. We want to estimate the mean and variance for `X` at a final timestep `n=n_steps`. 

```
           [N]
            |   
    X[n-1]  v  X[n]
   ------->[+]----->
```

We start by including the ForneyLab.jl toolbox and initializing these basic settings.

In [ ]:
# Load PyPlot and ForneyLab.jl
using PyPlot
using ForneyLab

# Basic settings
n_steps = 50  # Total number of timesteps.
m_noise = 0.0 # Mean of the noise, corresponds with a drift of X.
V_noise = 1.0 # Variance of the Gaussian noise.
X_0     = DeltaDistribution(0.0) # Initial setting for the state X at n=0;

The graph we will construct models one step in the random walk process. There are two ways to set up the nodes. Either initialize one timestep and feed the output back to the input, or build a chain by initializing all timesteps and connecting them to form the complete graph. We will choose the first option in this demo.

In [ ]:
# Initialize and connect nodes
X_n_min_1_node = PriorNode(X_0, name="X_n_min_1") # Initial state
X_n_node       = TerminalNode(name="X_n")
N_node         = PriorNode(GaussianDistribution(m=m_noise, V=V_noise), name="N") # Constant noise
add_node       = AdditionNode(name="add")

Edge(X_n_min_1_node, add_node.i[:in1])
Edge(N_node, add_node.i[:in2])
Edge(add_node.i[:out], X_n_node);

Now we need to indicate that the output at the right is actually the input for the next section. We do so by defining a `wrap()`:

In [ ]:
wrap(X_n_node, X_n_min_1_node);

Now when we visualize the graph, we notice the green arrow indicating the presence of wrap in time.

In [ ]:
draw()

The quantity we are interested in is the state of `x`. We can tell ForneyLab to store the online state estimate for each timepoint by initializing a write buffer.

In [ ]:
X = setWriteBuffer(add_node.i[:out]);

We also need to define an inference algorithm. With the write buffer set, `SumProduct.Algorithm()` will automatically initialize a sumproduct algorithm that passes messages towards the wraps and write buffers defined on the graph.

In [ ]:
algo = SumProduct.Algorithm()

Now our graph is ready for state estimation. We call the `step()` function on our algorithm `n_steps` times. Each call to `step()` executes the schedule, writes to the buffer and resets the graph for the next step.

In [ ]:
for n = 1:n_steps
    step(algo)
end

Our calculated state has been written to our buffer. We can extract the mean and standard deviation and plot the result.

In [ ]:
# Extract mean and standard deviation
X_means = [mean(X_n)[1] for X_n in [X_0, X]]
X_sds = [sqrt(var(X_n)[1,1]) for X_n in [X_0, X]]

# Make plot
plot([0:n_steps], X_means, "b-")
fill_between([0:n_steps], X_means-X_sds, X_means+X_sds, color="b", alpha=0.3)
xlabel("n")
ylabel("X[n]")
grid(true)

#Sampling a random walk path

Instead of estimating the expectation over all paths as in the previous section, we can also sample data from the model by drawing a sample after each step. For this we need to specify a `sample()` post-processing operation at the end of each section.

Note that we do not alter the graph structure but simply construct a new sumproduct algorithm with an altered schedule.

In [ ]:
# Define a new algorithm and edit the schedule entry that should draw a sample
clearMessages!()
algo2 = SumProduct.Algorithm()
schedule = algo2.fields[:schedule]
setPostProcessing!(schedule, X_n_node.i[:out].partner, sample) # Specify the sample function as post-processing
show(schedule)

In [ ]:
# Reset the starting point and write buffers
X_n_min_1_node.value = X_0
emptyWriteBuffers()

# Execute the mew schedule
for n = 1:n_steps
    step(algo2)
end

# Plot the samples
X_vals = [mean(X_n)[1] for X_n in X]
scatter(1:n_steps, X_vals)
xlabel("n")
ylabel("X[n]")
grid(true)